In [88]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


here the objective is to fit the logit graph model to multiple different graph domains and then visualize the original vs fitted graphs

In [89]:

import sys
import os
sys.path.append('../')

#Graph imports
import src.graph as graph
import src.logit_estimator as estimator
import src.utils as utils
import src.model_selection as model_selection
import src.gic as gic
import src.param_estimator as pe
import src.graph as graph
import src.model_selection as ms

# usual imports
import matplotlib.pyplot as plt
import pickle
import math
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import gaussian_kde
import numpy as np
import pandas as pd
import seaborn as sns
import gc
import random
import networkx as nx
from numpy import errstate

from IPython.display import display
from pyvis.network import Network
from mpl_toolkits.axes_grid1 import make_axes_locatable


In [91]:
FOLDER_OUTPUT = f'runs/fitted_graphs_comparison_facebook4'
out_files = os.listdir(FOLDER_OUTPUT)
out_files = sorted(out_files, key=lambda x: os.path.getctime(os.path.join(FOLDER_OUTPUT, x)), reverse=True)

out_files

['comparators_3437.pkl',
 'comparators_0.pkl',
 'comparators_348.pkl',
 'comparators_686.pkl',
 'comparators_414.pkl',
 'comparators_698.pkl',
 'comparators_3980.pkl']

In [92]:
import pickle
#pick first id from out_files
id = out_files[0].split('_')[1]

with open(f'{FOLDER_OUTPUT}/comparators_{id}', 'rb') as f:
    comparators = pickle.load(f)


In [93]:
import pickle
# with open(f'{FOLDER_OUTPUT}/fitted_graphs_698.pkl', 'rb') as f:
#     fitted_graphs = pickle.load(f)
# with open(f'{FOLDER_OUTPUT}/summary_dfs_698.pkl', 'rb') as f:
#     summary_dfs = pickle.load(f)

with open(f'{FOLDER_OUTPUT}/comparators_0.pkl', 'rb') as f:
    comparators = pickle.load(f)


In [94]:
len(comparators)

6

In [95]:
dfs = []
for i in range(len(comparators)):
    dfs.append(comparators[i].summary_df)

dfs[0]

,graph_filename,model,gic_value,param,fit_success,nodes,edges,density,avg_clustering,avg_path_length,diameter,assortativity,num_components,largest_component_size
0,3980.edges,Original,NaN,N/A,True,52,146,0.110106,0.461727,2.566596,5,0.160451,4,44
1,3980.edges,LG,1.841057,"d=0, sigma=-4.6308",True,52,152,0.114630,0.163551,2.377074,5,-0.102617,1,52
2,3980.edges,WS,0.470633,20.178889,True,52,156,0.117647,0.094231,2.347662,4,-0.078705,1,52
3,3980.edges,ER,0.574923,0.115556,True,52,147,0.110860,0.123876,2.424585,4,-0.106538,1,52
4,3980.edges,BA,0.582606,3.888889,True,52,147,0.110860,0.229728,2.322021,4,-0.160639,1,52
5,3980.edges,GRG,13.336563,1.0,True,52,1295,0.976621,0.978732,1.023379,2,-0.058264,1,52


In [96]:
pd.concat(dfs)

,graph_filename,model,gic_value,param,fit_success,nodes,edges,density,avg_clustering,avg_path_length,diameter,assortativity,num_components,largest_component_size
0,3980.edges,Original,NaN,N/A,True,52,146,0.110106,0.461727,2.566596,5,0.160451,4,44
1,3980.edges,LG,1.841057,"d=0, sigma=-4.6308",True,52,152,0.114630,0.163551,2.377074,5,-0.102617,1,52
2,3980.edges,WS,0.470633,20.178889,True,52,156,0.117647,0.094231,2.347662,4,-0.078705,1,52
3,3980.edges,ER,0.574923,0.115556,True,52,147,0.110860,0.123876,2.424585,4,-0.106538,1,52
4,3980.edges,BA,0.582606,3.888889,True,52,147,0.110860,0.229728,2.322021,4,-0.160639,1,52
5,3980.edges,GRG,13.336563,1.0,True,52,1295,0.976621,0.978732,1.023379,2,-0.058264,1,52
0,698.edges,Original,NaN,N/A,True,61,270,0.147541,0.733202,1.944872,4,0.287962,3,40
1,698.edges,LG,1.556714,"d=2, sigma=-3.7050",True,61,292,0.159563,0.152285,2.031148,4,0.059561,1,61
2,698.edges,ER,0.507371,0.2,True,61,346,0.189071,0.194985,1.893443,3,-0.013947,1,61
3,698.edges,BA,0.600498,5.0,True,61,280,0.153005,0.332798,1.996175,3,-0.214212,1,61


In [97]:
# Create a dictionary to store rankings for each graph
rankings = {}

# Go through each df in dfs
for df in dfs:
    graph_name = df['graph_filename'].iloc[0]  # Get graph filename
    
    # Sort models by GIC value (excluding 'Original' which has NaN GIC)
    model_ranks = df[df['model'] != 'Original'].sort_values('gic_value')['model'].tolist()
    
    # Create rankings dictionary for this graph (1 is best since lowest GIC is best)
    rankings[graph_name] = {model: rank+1 for rank, model in enumerate(model_ranks)}

# Get unique models (excluding 'Original')
models = sorted(list(set([model for df in dfs for model in df['model'].unique() if model != 'Original'])))

# Create DataFrame with rankings
ranking_df = pd.DataFrame(rankings).T

# Ensure all models are present as columns
for model in models:
    if model not in ranking_df.columns:
        ranking_df[model] = np.nan

# Keep only the model columns in a consistent order
ranking_df = ranking_df[models]

print("Rankings by GIC value (lower is better):")
display(ranking_df)


Rankings by GIC value (lower is better):


,BA,ER,GRG,LG,WS
3980.edges,3,2,5,4,1
698.edges,2,1,5,4,3
414.edges,1,2,5,3,4
686.edges,1,4,5,3,2
348.edges,2,3,5,4,1
0.edges,3,4,5,2,1


In [98]:
ranking_df.mean(axis=0)

BA     2.000000
ER     2.666667
GRG    5.000000
LG     3.333333
WS     2.000000
dtype: float64